In [1]:
import os

In [2]:
%pwd

'd:\\Imarticus_Learning\\12_Projects\\Phone_Recommendation_System\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Imarticus_Learning\\12_Projects\\Phone_Recommendation_System'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class DataExtractionConfig:
    root_dir: Path
    local_data_file: Path
    extracted_data_file: Path

In [6]:
from phone_recommender.constants import *
from phone_recommender.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH) -> None:
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

    def get_data_extraction_config(self) -> DataExtractionConfig:
        config = self.config.data_extraction

        create_directories([config.root_dir])

        data_extraction_config = DataExtractionConfig(
            root_dir=config.root_dir,
            local_data_file=config.local_data_file,
            extracted_data_file=config.extracted_data_file,
        )

        return data_extraction_config

In [8]:
import warnings
import pandas as pd
import numpy as np
import re
from phone_recommender.logging import logger

warnings.filterwarnings('ignore')

In [9]:
class DataExtraction:
    def __init__(self, config: DataExtractionConfig) -> None:
        self.config = config

    def get_local_data(self):
        local_data_file = self.config.local_data_file
        if not os.path.exists(local_data_file):
            logger.info("No local data file exists please check if data ingestion is complete!")
        else:
            df = pd.read_csv(local_data_file)
            return df

    def save_extracted_data(self, df: pd.DataFrame):
        extracted_data_file = self.config.extracted_data_file
        df.to_csv(extracted_data_file, index=False)

In [10]:
try:
    config = ConfigurationManager()
    data_extraction_config = config.get_data_extraction_config()
    data_extraction = DataExtraction(data_extraction_config)
    df = data_extraction.get_local_data()
except Exception as e:
    raise e

[2023-12-11 23:24:06,276: INFO: common: yaml file: config\config.yaml loads successfully]
[2023-12-11 23:24:06,278: INFO: common: yaml file: params.yaml loads successfully]
[2023-12-11 23:24:06,290: INFO: common: created directory at : artifacts/data_extraction]


In [11]:
df.head()

,brand,phone_name,text
0,huawei,nova 11 SE,Network Technology GSM / CDMA / HSPA / CDMA200...
1,huawei,Mate 60 Pro+,Network Technology GSM / CDMA / HSPA / CDMA200...
2,huawei,Mate X5,Network Technology GSM / CDMA / HSPA / EVDO / ...
3,huawei,Mate 60 Pro,Network Technology GSM / CDMA / HSPA / CDMA200...
4,huawei,nova Y91,Network Technology GSM / HSPA / LTE 2G bands G...


In [12]:
df.shape

(3019, 3)

### Data Extraction


In [13]:
df['text'] = df['text'].str.lower()

In [14]:
def get_network(string):
    res = re.search(r"network.*2g bands", string)
    if res is not None:
        network = re.sub(r"network technology|2g bands|\/|gsm", "", res.group()).strip()
        if ("lte" in network) or "5g" in network or "4g" in network:
            network = re.sub(r"cdma|hspa|cdma|cdma2000|evdo|2000|umts", "", network).strip()
            # print(network)
            return network
        else:
            # print("0")
            return "0"
    else:
        # print("0")
        return "0"

In [15]:
df['network'] = df['text'].apply(lambda x: get_network(x))

In [16]:
df[df['network'] == "0"].shape

(1200, 4)

#### Dropping values with little to no data


In [17]:
drop_idx = df[df['network'] == "0"].index
df = df.drop(index=drop_idx)
df.shape

(1819, 4)

In [18]:
df['text'][0]

'network technology gsm / cdma / hspa / cdma2000 / lte 2g bands gsm 850 / 900 / 1800 / 1900 - sim 1 & sim 2 cdma 800 3g bands hsdpa 850 / 900 / 1700(aws) / 1900 / 2100 cdma2000 1x 4g bands 1, 3, 4, 5, 8, 18, 19, 26, 34, 38, 39, 40, 41 speed hspa, lte-a launch announced 2023, october 31 status available. released 2023, november 03 body dimensions 162.4 x 75.5 x 7.4 mm (6.39 x 2.97 x 0.29 in) weight 186 g (6.56 oz) sim dual sim (nano-sim, dual stand-by) display type oled, 1b colors, 90hz size 6.67 inches, 107.4 cm2 (~87.6% screen-to-body ratio) resolution 1080 x 2400 pixels, 20:9 ratio (~395 ppi density) platform os harmonyos 4.0 chipset qualcomm snapdragon 680 4g (6 nm) cpu octa-core (4x2.4 ghz cortex-a73 & 4x1.9 ghz cortex-a53) gpu adreno 610 memory card slot no internal 256gb, 512gb main camera triple 108 mp, f/1.9, (wide), 1/1.67", 0.64µm, pdaf\n8 mp, f/2.2, 112˚ (ultrawide)\n2 mp, f/2.4, (macro) features led flash, hdr, panorama video 1080p@30fps selfie camera single 32 mp, f/2.5, (

#### Getting 4G Data


In [19]:
def get_4g_bands(text):
    pattern = re.compile(r"4g bands.*speed|4g bands.*5g")
    bands_4g = re.search(pattern, text)
    if bands_4g is not None:
        bands = re.sub(r"4g bands|speed|5g bands.*|", "", bands_4g.group()).strip()
        if "lte" not in bands:
            bands = re.findall(r"\d,|\d\d,", bands)
            if len(bands) not in [0, 1]:
                bands = " ".join(bands)
                bands = re.sub(",", "", bands)
                bands = " ".join([str(x) for x in sorted(([int(x) for x in set(bands.split())]))])
                # print(bands)
                return bands
            else:
                # print("1")
                return "1"
        else:
            # print("1")
            return "1"

In [20]:
df['4g_bands'] = df['text'].apply(lambda x: get_4g_bands(x))

In [21]:
def get_bands_count(text):
    if text not in ["1", "0"]:
        band_count = len(text.split())
        # print(band_count)
        return band_count
    elif text == "1":
        # print(1)
        return 1
    else:
        return 0

In [22]:
df['4g_band_count'] = df['4g_bands'].apply(lambda x: get_bands_count(x))

#### Getting 5G Data


In [23]:
def get_5g_bands(text):
    res = re.search(r"5g bands.*speed|5g bands.*announced", text)
    if res is not None:
        bands = re.sub("5g bands|speed", "", res.group()).strip()
        bands = re.sub(r"sa/nsa|sa/nsa/sub6", "1,", bands).strip()
        bands = re.findall(r"\d,|\d\d,|\d\d\d,", bands)
        if len(bands) not in [0, 1]:
            bands = " ".join(bands)
            bands = re.sub(",", "", bands)
            bands = " ".join([str(x) for x in sorted(([int(x) for x in set(bands.split())]))])
            # print(bands)
            return bands
        else:
            # print("1")
            return "1"
    else:
        # print("0")
        return "0"

In [24]:
df['5g_bands'] = df['text'].apply(lambda x: get_5g_bands(x))

In [25]:
df['5g_band_count'] = df['5g_bands'].apply(lambda x: get_bands_count(x))

#### Getting released year for phone


In [26]:
def get_released_year(text):
    pattern = re.compile(r"status.*dimensions")
    res = re.search(pattern=pattern, string=text)
    if res is not None:
        res = re.sub(r",|status|dimensions|available. released|body", "", res.group()).strip()
        if res not in ["discontinued", "cancelled"]:
            # print(res.split()[0])
            return res.split()[0]
        else:
            # print(res)
            return res
    else:
        # print('0')
        return "no info"

In [27]:
df['released_year'] = df['text'].apply(lambda x: get_released_year(x))

#### Phone Dimensions like height width and depth


In [28]:
def get_full_dimension(string):
    pattern = re.compile(r"dimensions.*weight")
    res = re.search(pattern=pattern, string=string).group()
    if res is not None:
        res = re.sub(r"dimensions|weight|mm.*|unfolded|or.*|cc|\(.*|-.*|:|x", "", res).strip()
        if len(res.split()) == 3:
            # print(res)
            return " ".join(res.split())
        else:
            # print("0 x 0 x 0")
            return "0 0 0"
    else:
        # print("0 x 0 x 0")
        return "0 0 0"

In [29]:
df['height_width_depth'] = df['text'].apply(lambda x: get_full_dimension(x))

In [30]:
def get_hwd(text):
    res = [0.0, 0.0, 0.0] if text == "0 0 0" else [round(float(x), 1) for x in text.split()]
    return pd.Series(res, index=['height', 'width', 'depth'])

In [31]:
df[['height', 'width', 'depth']] = df['height_width_depth'].apply(lambda x: get_hwd(x))

#### Weight of the phone


In [32]:
def get_weight(string):
    res = re.search(r"weight.*sim|weight.*build", string)
    if res is not None:
        weight = re.search(r"\d\d\d g|\d\d g", res.group())
        if weight is not None:
            weight = re.sub(r"g", "", weight.group()).strip()
            return weight
        else:
            # print("0")
            return "0"
    else:
        # print("0")
        return "0"

In [33]:
df['weight'] = df['text'].apply(lambda x: get_weight(x))

#### Phone display specifications


In [34]:
def get_resolution(string):
    res = re.search(r"resolution.*protection|resolution.*os", string)
    if res is not None:
        resolution = re.search(r"resolution.*pixels", res.group())
        ppi = re.search(r"\(~.* ppi", res.group())
        if resolution and ppi is not None:
            resolution = " ".join(re.sub(r",.*|\(.*|pixels.*|resolution|:|x", "", resolution.group()).strip().split())
            ppi = re.sub(r"\(~|ppi", "", ppi.group()).strip()
            # print([resolution, ppi])
            return pd.Series([resolution, ppi], index=['resolution', 'ppi'])
        else:
            # print(["0 0", "0 ppi"])
            return pd.Series(["0 0", "0"], index=['resolution', 'ppi'])
    else:
        # print(["0 0", "0 ppi"])
        return pd.Series(["0 0", "0"], index=['resolution', 'ppi'])

In [35]:
df[['resolution', 'ppi']] = df['text'].apply(lambda x: get_resolution(x))

In [36]:
def get_resolution_wh(resolution):
    res = resolution.split()
    return pd.Series([float(res[0]), float(res[1])], index=['display_width', 'display_height'])

In [37]:
df[['display_width', 'display_height']] = df['resolution'].apply(lambda x: get_resolution_wh(x))

In [38]:
def get_ppi(ppi):
    res = ppi
    return float(res)

In [39]:
df['display_ppi'] = df['ppi'].apply(lambda x: get_ppi(x))

In [40]:
def display_size_str(string):
    res = re.search(r"size.*inches,", string)
    if res is not None:
        res = re.sub(r"size|inches,.*", "", res.group()).strip()
        res = str(round(float(res), 1))
        # print(res)
        return res
    else:
        # print("0")
        return "0"

In [41]:
df['display_size_str'] = df['text'].apply(lambda x: display_size_str(x))

In [42]:
def display_size(string):
    res = re.search(r"size.*inches,", string)
    if res is not None:
        res = re.sub(r"size|inches,.*", "", res.group()).strip()
        res = round(float(res), 1)
        # print(res)
        return res
    else:
        # print("0")
        return 0

In [43]:
df['display_size'] = df['text'].apply(lambda x: display_size(x))

In [44]:
def display_type(text):
    res = re.search(r"display type.*size", text)
    if res is not None:
        display_type = re.sub(
            r"display type|size|,|\(|\)|2x|international.*|hdr10|\+|ical|plus|hdr|[0-9bk]{2,4} colors|flexible|dynamic|foldable|[0-9]{3,4} nits|dolby vision|typ|hbm|peak",
            "",
            res.group(),
        ).strip()
        display_type = re.sub(r"amoled", "oled", display_type).strip()
        display_type = " ".join(display_type.split())
        # print(display_type)
        return display_type
    else:
        # print("no info")
        return "no info"

In [45]:
df['display_type'] = df['text'].apply(lambda x: display_type(x))

#### Phone Operating system


In [46]:
def get_os(text):
    res = re.search(r"os.*chipset", text)
    if res is not None:
        platform = re.sub(r"os|chipset|oreo|pie|or.*|,.*|\(.*|\/.*", "", res.group()).strip()
        if platform.split()[0] in ["5433)hspa", "t"]:
            platform = re.sub(r"5433\)hspa 42\.2\/5\.76 mbps|t.*", "0", platform)
        # platform = re.sub(r"5433)hspa 42.2/5.76 mbps", "0", platform).strip()
        # print(platform)
        return "".join(platform.split())
    else:
        # print("0")
        return "0"

In [47]:
df['os'] = df['text'].apply(lambda x: get_os(x))

#### Phone Chipset / Processor


In [48]:
def get_chipset(text):
    res = re.search(r"chipset.*", text)
    if res is not None:
        chipset = re.search(
            r"kirin [0-9a-z]{2,5}|snapdragon [0-9+gens ]{2,8}|dimensity [0-9]{2,5}|exynos [0-9]{2,4}|helio [0-9pgax]{1,4}",
            res.group(),
        )
        if chipset is not None:
            chipset = re.sub(r" 5g| 4g", "", chipset.group())
            # print(chipset)
            return "".join(chipset.split())
        else:
            # print("0")
            return "0"
    else:
        # print("0")
        return "0"

In [49]:
df['chipset'] = df['text'].apply(lambda x: get_chipset(x))

#### Phone Ram and Storage


In [50]:
def get_memory(text):
    res = re.search(r"memory.*main camera", text)
    if res is not None:
        res = re.sub(r".*internal|main.*|\(.*\)|- india|\.x|\d+mb", "", res.group())
        ram = " ".join(np.unique(re.findall(r"\dgb ram|\d\dgb ram", res)))
        stg = re.sub(r"\dgb ram|\d\dgb ram|,|1\.|sfs|umcp|emcp|", "", res).strip()
        ram = re.sub(r"ram", "", ram)
        type = re.search(r"ufs.*|emmc.*", stg)
        if type is not None:
            stg = re.sub(r"ufs.*|emmc.*|sfs|umcp", "", stg)
            stg = " ".join(np.unique(stg.split()))
            type = re.sub(r"or.*", "", "".join(type.group().strip().split()))
            # print(ram, stg, type)
            return pd.Series([ram, stg, type], index=["ram", "storage", "type"])
        elif (ram == "") and (stg != ""):
            # print(f"0 {stg} 0")
            return pd.Series(["0", stg, "0"], index=["ram", "storage", "type"])
        else:
            stg = " ".join(np.unique(stg.split()))
            # print(ram, stg)
            return pd.Series([ram, stg, "0"], index=["ram", "storage", "type"])
    else:
        # print("0 0 0")
        return pd.Series(["0", "0", "0"], index=["ram", "storage", "type"])

In [51]:
# df['text'].apply(lambda x: get_memory(x))
df[["ram", "storage", "type"]] = df['text'].apply(lambda x: get_memory(x))

#### Back camera


In [52]:
def get_main_camera(text):
    res = re.search(r"main camera.*", text)
    if res is not None:
        main_camera = re.sub(
            r"main camera|selfie.*|,|\/|\(|\)|\"|triple|dual|single|feature|video|quad", "", res.group()
        ).strip()
        main_camera = re.search(r"\d mp|\d\d mp|\d\d\d mp", main_camera)
        if main_camera is not None:
            # print(main_camera.group())
            return "".join(main_camera.group().strip().split())
        else:
            # print("0")
            return "0"
    else:
        # print('0')
        return '0'

In [53]:
df['main_camera'] = df['text'].apply(lambda x: get_main_camera(x))

#### Front camera


In [54]:
def get_selfie_camera(text):
    res = re.search(r"selfie camera.*", text)
    if res is not None:
        selfie_camera = " ".join(
            re.sub(r"sound.*|selfie camera|single|dual|\/|\(|\)|\"|,|-|video", "", res.group()).strip().split()
        )
        selfie_camera = re.search(r"\d mp|\d\d mp|\d\d\d mp", selfie_camera)
        if selfie_camera is not None:
            # print(selfie_camera.group())
            return "".join(selfie_camera.group().strip().split())
        else:
            # print("0")
            return "0"
    else:
        # print("0")
        return "0"

In [55]:
df['selfie_camera'] = df['text'].apply(lambda x: get_selfie_camera(x))

#### Bluetooth Info


In [56]:
def get_comms(text):
    res = re.search(r"comms.*", text)
    if res is not None:
        comms = re.sub(r"positioning.*|,|dual-band|comms|wlan|wi-fi|wi-fi direct|a2dp|le", "", res.group()).strip()
        bluetooth = re.search(r"bluetooth [0-9.yesno]{1,4}", comms)
        if bluetooth is not None:
            # print(bluetooth.group())
            return "".join(bluetooth.group().strip().split())
        else:
            # print("0")
            return "0"
    else:
        # print('0')
        return '0'

In [57]:
df['bluetooth'] = df['text'].apply(lambda x: get_comms(x))

#### Battery Size


In [58]:
def get_battery(text):
    battery_mah = re.search(r"\d\d\d\d mah", text)
    if battery_mah is not None:
        # print(battery_mah.group())
        return "".join(battery_mah.group().strip().split())
    else:
        # print("0")
        return "0"

In [59]:
df['battery'] = df['text'].apply(lambda x: get_battery(x))

In [60]:
def get_price(text):
    price = re.search(r"price.*", text)
    if price is not None:
        price = re.search(r"\d\d\d eur|\d\d eur|€ [0-9.,]{2,10}|₹ [0-9,]{2,10}", price.group())
        if price is not None:
            return price.group().strip()
        else:
            return "0"
    else:
        return "0"

In [61]:
df['price'] = df['text'].apply(lambda x: get_price(x))

In [62]:
def get_price_int(price):
    if "₹" in price:
        price = re.sub(r"₹|,", "", price)
        # print(round(float(price), 0))
        return round(float(price), 0)
    elif "€" in price:
        price = re.sub(r"€|,", "", price)
        # print(round(float(price), 0) * 89)
        return round(float(price), 0) * 89
    elif "eur" in price:
        price = re.sub(r"eur|,", "", price)
        # print(round(float(price), 0) * 89)
        return round(float(price), 0) * 89
    else:
        # print(0)
        return 0

In [63]:
df['price'] = df['price'].apply(lambda x: get_price_int(x))

In [64]:
df.head()

,brand,phone_name,text,network,4g_bands,4g_band_count,5g_bands,5g_band_count,released_year,height_width_depth,...,os,chipset,ram,storage,type,main_camera,selfie_camera,bluetooth,battery,price
0,huawei,nova 11 SE,network technology gsm / cdma / hspa / cdma200...,lte,1 3 4 5 8 18 19 26 34 38 39 40,12,0,0,2023,162.4 75.5 7.4,...,harmony4.0,snapdragon680,0,256gb 512gb,0,108mp,32mp,bluetooth5.0,4500mah,23140.0
1,huawei,Mate 60 Pro+,network technology gsm / cdma / hspa / cdma200...,lte 5g,1,1,1,1,2023,163.7 79 8.1,...,harmony4.0,kirin9000s,16gb,1tb 512gb,ufs,48mp,13mp,bluetooth5.2,5000mah,0.0
2,huawei,Mate X5,network technology gsm / cdma / hspa / evdo / ...,lte 5g,1,1,1,1,2023,156.9 141.5 5.3,...,harmony4.0,kirin9000s,12gb 16gb,1tb 256gb 512gb,ufs,50mp,8mp,bluetooth5.2,5060mah,0.0
3,huawei,Mate 60 Pro,network technology gsm / cdma / hspa / cdma200...,lte 5g,1,1,1,1,2023,163.7 79 8.1,...,harmony4.0,kirin9000s,12gb,1tb 256gb 512gb,ufs,50mp,13mp,bluetooth5.2,5000mah,79210.0
4,huawei,nova Y91,network technology gsm / hspa / lte 2g bands g...,lte,1 2 3 4 5 7 8 13 20 26 28 38 40 41,14,0,0,2023,171.6 79.9 8.9,...,emui13,snapdragon680,6gb 8gb,128gb 256gb,0,50mp,8mp,bluetooth5.0,7000mah,24920.0


### Saving Extracted Data


In [65]:
data_extraction.save_extracted_data(df=df)

### Finished
